In [25]:
from time import sleep, strftime
from random import randint
import pandas as pd
from selenium import webdriver
#from selenium.webdriver.commom.keys import Keys
import smtplib
from email.mime.multipart import MIMEMultipart

In [26]:
chromedriver_path = 'C:\\Users\\hp\\.wdm\\drivers\\chromedriver\\81.0.4044.69\\win32\\chromedriver.exe'

In [27]:
driver = webdriver.Chrome('./chromedriver')
sleep(2)

In [28]:
kayak = 'https://www.kayak.co.in/flights/BLR-BOM/2020-09-28-flexible/2020-09-28-flexible?sort=bestflight_a'
driver.get(kayak)
sleep(3)

In [29]:
cheap_results = '//a[@data-code="price"]'
driver.find_element_by_xpath(cheap_results).click()

In [30]:
xp_results_table = '//*[@class="resultWrapper"]'
flight_containers = driver.find_elements_by_xpath(xp_results_table)
flights_list = [flight.text for flight in flight_containers]

flights_list[0:3]

['COVID-19 policies\n28/9\nMon\n05:45 – 07:25\nIndiGo\ndirect\n1h 40m\nBLR\n‐\nBOM\n28/9\nMon\n10:30 – 12:20\nAir India\ndirect\n1h 50m\nBOM\n‐\nBLR\nSponsored result\n₹ 8,624\neDreams\nView Deal',
 'Cheapest\nCOVID-19 policies\n28/9\nMon\n05:30 – 07:10\nAirAsia India\ndirect\n1h 40m\nBLR\n‐\nBOM\n28/9\nMon\n19:10 – 20:55\nVistara\ndirect\n1h 45m\nBOM\n‐\nBLR\n₹ 7,837\nKiwi.com\n₹ 7,881\nGotogate\n1 more\n₹ 6,556\nHappyEasyGo\nView Deal',
 'Best\nRating:\n10\n28/9\nMon\n05:30 – 07:10\nAirAsia India\ndirect\n1h 40m\nBLR\n‐\nBOM\n28/9\nMon\n18:30 – 20:05\nAirAsia India\ndirect\n1h 35m\nBOM\n‐\nBLR\n₹ 6,853\nAgoda.com\n₹ 7,090\nGotogate\n4 more\n₹ 6,557\nHappyEasyGo\nView Deal']

In [31]:
def load_more():
    try:
        more_results = '//a[@class="moreButton"]'
        driver.find_element_by_xpath(more_results).click()
        print('sleeping.....')
        sleep(randint(45,60))
    except:
        pass

In [32]:
def page_scrape():
    
    xp_sections = '//*[@class="section duration allow-multi-modal-icons"]'
    sections = driver.find_elements_by_xpath(xp_sections)
    sections_list = [value.text for value in sections]
    section_a_list = sections_list[::2]
    section_b_list = sections_list[1::2]
    
    
    if section_a_list == []:
        raise SystemExit
    
    
    a_duration = []
    a_section_names = []
    for n in section_a_list:
        a_section_names.append(''.join(n.split()[2:5]))
        a_duration.append(''.join(n.split()[0:2]))
    
   
    b_duration = []
    b_section_names = []
    for n in section_b_list:
        b_section_names.append(''.join(n.split()[2:5]))
        b_duration.append(''.join(n.split()[0:2]))
        
    
    xp_dates = '//div[@class="section date"]'
    dates = driver.find_elements_by_xpath(xp_dates)
    dates_list = [value.text for value in dates]
    a_dates_list = dates_list[::2]
    b_dates_list = dates_list[1::2]
    
    a_day = [value.split()[0] for value in a_dates_list]
    a_weekday = [value.split()[1] for value in a_dates_list]
    b_day = [value.split()[0] for value in b_dates_list]
    b_weekday = [value.split()[1] for value in a_dates_list]
    
    xp_prices = '//a[@class="booking-link "]/span[@class="price option-text"]'
    prices = driver.find_elements_by_xpath(xp_prices)
    price_list = [price.text.replace('₹','') for price in prices if price.text!='']
    price_list = [price.replace(',','') for price in price_list]
    #print(price_list[0:5])
    price_list = list(map(int,price_list))
    
    xp_stops = '//div[@class="section stops"]/div[1]'
    stops = driver.find_elements_by_xpath(xp_stops)
    stops_list = [value.text[0].replace('n','0') for value in stops]
    a_stop_list = stops_list[::2]
    b_stop_list = stops_list[1::2]
    
    xp_stops_cities = '//div[@class="section stops"]/div[2]'
    stops_cities = driver.find_elements_by_xpath(xp_stops_cities)
    stops_cities_list = [stop.text for stop in stops_cities]
    a_stop_name_list = stops_cities_list[::2]
    b_stop_name_list = stops_cities_list[1::2]
    
    xp_schedule = '//div[@class="section times"]'
    schedules = driver.find_elements_by_xpath(xp_schedule)
    hours_list = []
    carrier_list = []
    for schedule in schedules:
        hours_list.append(schedule.text.split('\n')[0])
        carrier_list.append(schedule.text.split('\n')[1])
        
    a_hours = hours_list[::2]
    a_carrier = carrier_list[::2]
    b_hours = hours_list[1::2]
    b_carrier = carrier_list[1::2]
    
    
    cols = (['Out Day', 'Out Time', 'Out Weekday', 'Out Airline', 'Out Cities', 'Out Duration', 'Out Stops', 'Out Stop Cities',
            'Return Day', 'Return Time', 'Return Weekday', 'Return Airline', 'Return Cities', 'Return Duration', 'Return Stops', 'Return Stop Cities',
            'Price'])
    
    flights_df = pd.DataFrame({'Out Day': a_day,
                               'Out Weekday': a_weekday,
                               'Out Duration': a_duration,
                               'Out Cities': a_section_names,
                               'Return Day': b_day,
                               'Return Weekday': b_weekday,
                               'Return Duration': b_duration,
                               'Return Cities': b_section_names,
                               'Out Stops': a_stop_list,
                               'Out Stop Cities': a_stop_name_list,
                               'Return Stops': b_stop_list,
                               'Return Stop Cities': b_stop_name_list,
                               'Out Time': a_hours,
                               'Out Airline': a_carrier,
                               'Return Time': b_hours,
                               'Return Airline': b_carrier,                           
                               'Price': price_list})[cols]
    
    flights_df['timestamp'] = strftime("%Y%m%d-%H%M")  #to know when it was scraped
    return flights_df

In [33]:
def start_kayak(city_from,city_to,date_start,date_end):
    """City codes - it's the IATA codes!
    Date format -  YYYY-MM-DD"""
    
    kayak = ('https://www.kayak.co.in/flights/' + city_from + '-' + city_to + '/' + date_start + '-flexible/' + date_end + '-flexible?sort=bestflight_a')
    driver.get(kayak)
    sleep(randint(8,10))
    
    
    try:
        xp_popup_close = '//button[contains(@id,"dialog-close") and contains(@class,"Button-No-Standard-Style close ")]'
        driver.find_elements_by_xpath(xp_popup_close)[5].click()
    except Exception as e:
        pass
    sleep(randint(60,95))
    print('loading more.....')
    
    print('starting first scrape.....')
    df_flights_best = page_scrape()
    df_flights_best['sort'] = 'best'
    sleep(randint(60,80))
    
    matrix = driver.find_elements_by_xpath('//*[contains(@id,"FlexMatrixCell")]')
    matrix_prices = [price.text.replace("₹",'') for price in matrix if price.text!='']
    matrix_prices = [price.replace(',','') for price in matrix_prices]
    matrix_prices = list(map(int,matrix_prices))
    matrix_min = min(matrix_prices)
    matrix_avg = sum(matrix_prices)/len(matrix_prices)
    
    print("Switching to cheapest results.....")
    cheap_results = '//a[@data-code="price"]'
    driver.find_element_by_xpath(cheap_results).click()
    sleep(randint(60,90))
    print("loading more.....")
    
    print('starting second scrape.....')
    df_flights_cheap = page_scrape()
    df_flights_cheap['sort'] = 'cheap'
    sleep(randint(60,80))
    
    print("Switching to quickest results.....")
    quick_results = '//a[@data-code="duration"]'
    driver.find_element_by_xpath(quick_results).click()
    sleep(randint(60,90))
    print("loading more.....")
    
    print('starting third scrape.....')
    df_flights_quick = page_scrape()
    df_flights_quick['sort'] = 'fast'
    sleep(randint(60,80))
    
    
    final_df = df_flights_cheap.append(df_flights_best).append(df_flights_quick)
    final_df.to_excel('Desktop//{}_flights_{}-{}_from_{}_to_{}.xlsx'.format(strftime("%Y%m%d-%H%M"),city_from,city_to,date_start,date_end),index = False)
    
    print("saved df.....")
    
    xp_loading = '//div[contains(@id,"advice")]'
    loading = driver.find_element_by_xpath(xp_loading).text
    xp_prediction = '//span[@class="info-text"]'
    prediction = driver.find_element_by_xpath(xp_prediction).text
    print(loading+'\n'+prediction)
    
    weird = '¯\\_(ツ)_/¯'
    if loading == weird:
        loading = 'Not sure'
        
    username = 'siddharth555@outlook.com'
    password = 'Siddharth1999'
    
    server = smtplib.SMTP('smtp.outlook.com',587)
    server.ehlo()
    server.starttls()
    server.login(username, password)
    msg = ('Subject: Flight Scraper\n\n\
           Cheapest Flight: {}\nAverage Price: {}\n\nRecommendation: {}\n\nEnd of message'.format(matrix_min, matrix_avg, (loading+'\n'+prediction)))
    message = MIMEMultipart()
    message['From'] = 'siddharth555@outlook.com'
    message['to'] = 'siddharth.krao99@gmail.com'
    server.sendmail('siddharth555@outlook.com', 'siddharth.krao99@gmail.com', msg)
    print('sent email.....')

In [34]:
city_from = input('From which city? ')
city_to = input('Where to? ')
date_start = input('Search around which departure date? Please use YYYY-MM-DD format only ')
date_end = input('Return when? Please use YYYY-MM-DD format only ')

# city_from = 'LIS'
# city_to = 'SIN'
# date_start = '2019-08-21'
# date_end = '2019-09-07'

for n in range(0,5):
    start_kayak(city_from, city_to, date_start, date_end)
    print('iteration {} was complete @ {}'.format(n, strftime("%Y%m%d-%H%M")))
    
    # Wait 4 hours
    sleep(60*60*4)
    print('sleep finished.....')

From which city? Bangalore
Where to? Mumbai
Search around which departure date? Please use YYYY-MM-DD format only 2020-09-28
Return when? Please use YYYY-MM-DD format only 2020-09-30
loading more.....
starting first scrape.....


ValueError: arrays must all be same length